In [2]:
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt
import numpy as np
plt.style.use('seaborn-dark')
import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split

pd.set_option('display.max_columns', None)
import matplotlib as plt
plt.rc('font', size=14)

def plot_2d_separator(classifier, X, fill=False, line=True, ax=None, eps=None):
    if eps is None:
        
        eps = 1.0
    x_min, x_max = X[:, 0].min() - eps, X[:, 0].max() + eps
    y_min, y_max = X[:, 1].min() - eps, X[:, 1].max() + eps
    xx = np.linspace(x_min, x_max, 100)
    yy = np.linspace(y_min, y_max, 100)
    
    X1, X2 = np.meshgrid(xx, yy)
    X_grid = np.c_[X1.ravel(), X2.ravel()]
    try:
        decision_values = classifier.decision_function(X_grid)
        levels = [0]
        fill_levels = [decision_values.min(), 0, decision_values.max()]
    except:
        decision_values = classifier.predict_proba(X_grid)[:, 1]
        levels = [.5]
        fill_levels = [0, .5, 1]
        
    if ax is None:
        ax = plt.gca()
    if fill:
        ax.contourf(X1, X2, decision_values.reshape(X1.shape),
                    levels=fill_levels, colors=['cyan', 'pink'])
    if line:
        ax.contour(X1, X2, decision_values.reshape(X1.shape), levels=levels,
                   colors="black")
    ax.set_xlim(x_min, x_max)
    ax.set_ylim(y_min, y_max)
    ax.set_xticks(())
    ax.set_yticks(())

## Загружаем данные

train = pd.read_csv('/home/paniquex/PycharmProjects/IML_homeworks/credit_scoring/train.csv')
test = pd.read_csv('/home/paniquex/PycharmProjects/IML_homeworks/credit_scoring/test.csv')
print(train['плохой_клиент'][train['семья'].isna()].value_counts())
print(train['плохой_клиент'].value_counts())
print(train.shape, test.shape)

y = train.pop('плохой_клиент')
train.shape, y.shape

## Предобработка

## Очень много пропусков в доходе: 

print(train.columns)
print(train.info())

print(test.info())

## Интересный человек №21595

train.sort_values(by='недвижимость', ascending=False).head()

train.describe()

train['доход'][train["доход"] < 1].value_counts()

## Заполним пропуски в доходе средним значением по всей таблице, а значение семьи = 0 

train.sort_values(by='доход', ascending=False).head(10)

mean_income_train = train['доход'].mean()
mean_income_test = test['доход'].mean()
print(mean_income_train, mean_income_test)
train['доход'].fillna(mean_income_train, inplace=True)
test['доход'].fillna(mean_income_test, inplace=True)

### Семья

train['семья'].fillna(1, inplace=True)
test['семья'].fillna(1, inplace=True)

0    2815
1     136
Name: плохой_клиент, dtype: int64
0    104956
1      7544
Name: плохой_клиент, dtype: int64
(112500, 11) (37500, 10)
Index(['линии', 'возраст', 'поведение_30-59_дней', 'Debt_Ratio', 'доход',
       'число_кредитов', 'поведение_90_дней', 'недвижимость',
       'поведение_60-89_дней', 'семья'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112500 entries, 0 to 112499
Data columns (total 10 columns):
линии                   112500 non-null float64
возраст                 112500 non-null int64
поведение_30-59_дней    112500 non-null int64
Debt_Ratio              112500 non-null float64
доход                   90274 non-null float64
число_кредитов          112500 non-null int64
поведение_90_дней       112500 non-null int64
недвижимость            112500 non-null int64
поведение_60-89_дней    112500 non-null int64
семья                   109549 non-null float64
dtypes: float64(4), int64(6)
memory usage: 8.6 MB
None
<class 'pandas.core.frame.DataF

In [3]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict

class DjStacking(BaseEstimator, ClassifierMixin):  
    """Стэкинг моделей scikit-learn"""

    def __init__(self, models, ens_model):
        """
        Инициализация
        models - базовые модели для стекинга
        ens_model - мета-модель
        """
        self.models = models
        self.ens_model = ens_model
        self.n = len(models)
        self.valid = None
        
    def fit(self, X, y=None, p=0.25, cv=3, err=0.001, random_state=None):
        """
        Обучение стекинга
        p - в каком отношении делить на обучение / тест
            если p = 0 - используем всё обучение!
        cv  (при p=0) - сколько фолдов использовать
        err (при p=0) - величина случайной добавки к метапризнакам
        random_state - инициализация генератора
            
        """
        if (p > 0): # делим на обучение и тест
            # разбиение на обучение моделей и метамодели
            train, valid, y_train, y_valid = train_test_split(X, y, test_size=p, random_state=random_state)
            
            # заполнение матрицы для обучения метамодели
            self.valid = np.zeros((valid.shape[0], self.n))
            for t, clf in enumerate(self.models):
                clf.fit(train, y_train)
                self.valid[:, t] = clf.predict(valid)
                
            # обучение метамодели
            self.ens_model.fit(self.valid, y_valid)
            
        else: # используем всё обучение
            
            # для регуляризации - берём случайные добавки
            self.valid = err*np.random.randn(X.shape[0], self.n)
            
            for t, clf in enumerate(self.models):
                # это oob-ответы алгоритмов
                self.valid[:, t] += cross_val_predict(clf, X, y, cv=cv, n_jobs=-1, method='predict')
                # но сам алгоритм надо настроить
                clf.fit(X, y)
            
            # обучение метамодели
            self.ens_model.fit(self.valid, y)  
            

        return self
    


    def predict(self, X, y=None):
        """
        Работа стэкинга
        """
        # заполение матрицы для мета-классификатора
        X_meta = np.zeros((X.shape[0], self.n))
        
        for t, clf in enumerate(self.models):
            X_meta[:, t] = clf.predict(X)
        
        a = self.ens_model.predict(X_meta)
        
        return (a)

SyntaxError: invalid syntax (<ipython-input-4-749271619767>, line 4)